In [1]:
%pip install requests

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import json
import os

In [3]:
#Define path
SECRETS_FILE = '/Users/gmccurdy/repos/curriculum/scripts/utilities/secrets.json'

# Load secrets
with open(SECRETS_FILE, 'r') as file:
    secrets = json.load(file)

# Assign variables from the secrets
API_URL = secrets.get('API_URL')
API_KEY = secrets.get('API_KEY')
COURSE_ID = secrets.get('COURSE_ID_APSTATISTICS')

#Define headers
headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

In [4]:
# Create the quiz
quiz_title = input('Quiz Title')
quiz_description = input('Quiz Description')
quiz_data = {
    'quiz': {
        'title': quiz_title,
        'description': f'<p>{quiz_description}</p>',
        'quiz_type': 'assignment',
        'published': False
    }
}

create_quiz_url = f'{API_URL}/api/v1/courses/{COURSE_ID}/quizzes'
response = requests.post(create_quiz_url, headers=headers, json=quiz_data)

if response.status_code == 200:
    quiz = response.json()
    quiz_id = quiz['id']
    print(f'Quiz created successfully with ID: {quiz_id}')
else:
    print('Failed to create quiz')
    print(f"Status Code: {response.status_code}")
    print(response.text)

Quiz created successfully with ID: 44071


In [5]:
multiple_choice_questions = [
    {
        'question_name': 'Critical Value for 95% Confidence Level',
        'question_type': 'multiple_choice_question',
        'question_text': '''What is the critical value \( z^* \) for a 95% confidence interval?''',
        'points_possible': 1,
        'answers': [
            {'text': '1.645', 'weight': 0},
            {'text': '1.960', 'weight': 100},  # Correct answer
            {'text': '2.576', 'weight': 0},
            {'text': '2.326', 'weight': 0},
            {'text': '1.282', 'weight': 0}
        ]
    },
    {
        'question_name': 'Precondition for Z-Interval for Proportion',
        'question_type': 'multiple_choice_question',
        'question_text': '''Which of the following is NOT a precondition for constructing a 95% confidence interval for a population proportion using a z-interval?''',
        'points_possible': 1,
        'answers': [
            {'text': 'Random Sampling', 'weight': 0},
            {'text': 'Independence of Observations', 'weight': 0},
            {'text': 'Success-Failure Condition: \( n\hat{p} \geq 10 \) and \( n(1 - \hat{p}) \geq 10 \)', 'weight': 0},
            {'text': 'Population Standard Deviation is Known', 'weight': 100},  # Not applicable for proportions
            {'text': 'Normality of the Sampling Distribution', 'weight': 0}
        ]
    },
    {
        'question_name': 'Standard Error for Population Proportion',
        'question_type': 'multiple_choice_question',
        'question_text': '''What is the formula for the standard error (SE) when constructing a confidence interval for a population proportion \( p \)?''',
        'points_possible': 1,
        'answers': [
            {'text': r'$SE = \frac{\sigma}{\sqrt{n}}$', 'weight': 0},
            {'text': r'$SE = \frac{s}{\sqrt{n}}$', 'weight': 0},
            {'text': r'$SE = \sqrt{\frac{\hat{p}(1 - \hat{p})}{n}}$', 'weight': 100},  # Correct answer
            {'text': r'$SE = \hat{p} \times (1 - \hat{p})$', 'weight': 0},
            {'text': r'$SE = \frac{\hat{p}}{n}$', 'weight': 0}
        ]
    },
    {
        'question_name': 'Margin of Error Calculation',
        'question_type': 'multiple_choice_question',
        'question_text': '''If the standard error (SE) is 2 and the critical value \( z^* \) is 1.96, what is the margin of error (ME) for the confidence interval?''',
        'points_possible': 1,
        'answers': [
            {'text': '1.96', 'weight': 0},
            {'text': '3.92', 'weight': 100},  # Correct answer: ME = z* × SE = 1.96 × 2 = 3.92
            {'text': '0.98', 'weight': 0},
            {'text': '4.88', 'weight': 0},
            {'text': '2.00', 'weight': 0}
        ]
    },
    {
        'question_name': 'Point Estimate of Confidence Interval',
        'question_type': 'multiple_choice_question',
        'question_text': '''In the confidence interval \( 0.564, 0.636 \), what is the point estimate?''',
        'points_possible': 1,
        'answers': [
            {'text': '0.564', 'weight': 0},
            {'text': '0.636', 'weight': 0},
            {'text': '0.6', 'weight': 100},    # Correct answer: Point estimate is the sample proportion, which is the midpoint
            {'text': '0.072', 'weight': 0},
            {'text': '1.200', 'weight': 0}
        ]
    },
    {
        'question_name': 'Identifying Appropriate Confidence Interval Type',
        'question_type': 'multiple_choice_question',
        'question_text': '''A researcher has collected a random sample of 100 data points and knows the population standard deviation. She wants to estimate the population mean with a 99% confidence interval. Which type of confidence interval should she use?''',
        'points_possible': 1,
        'answers': [
            {'text': 'T-Interval', 'weight': 0},
            {'text': 'Z-Interval', 'weight': 100},  # Correct answer
            {'text': 'Chi-Square Interval', 'weight': 0},
            {'text': 'F-Interval', 'weight': 0},
            {'text': 'Bootstrap Interval', 'weight': 0}
        ]
    },
    {
        'question_name': 'Identifying Appropriate Confidence Interval Type (Word Problem)',
        'question_type': 'multiple_choice_question',
        'question_text': '''A school administrator wants to estimate the average test score of all 9th-grade students. She randomly samples 40 students and calculates the sample mean and standard deviation. The population standard deviation is unknown. Which type of confidence interval should she use to estimate the average test score with 95% confidence?''',
        'points_possible': 1,
        'answers': [
            {'text': 'Z-Interval', 'weight': 0},
            {'text': 'T-Interval', 'weight': 100},  # Correct answer
            {'text': 'Chi-Square Interval', 'weight': 0},
            {'text': 'F-Interval', 'weight': 0},
            {'text': 'Bootstrap Interval', 'weight': 0}
        ]
    }
]




In [6]:
essay_questions = [

]

In [7]:
numerical_questions = [
    {
        'question_name': 'Calculating Standard Error for Mean',
        'question_type': 'numerical_question',
        'question_text': '''A sample of 64 individuals has a sample mean height of 170 cm. The population standard deviation is known to be 8 cm. Calculate the standard error (SE) for the 95% confidence interval.''',
        'points_possible': 1,
        'answers': [
            {
                'numerical_answer_type': 'exact_answer',
                'answer_exact': '1.0000'  # SE = 8 / sqrt(64) = 1.0
            }
        ]
    },
    {
        'question_name': 'Calculating Length of Confidence Interval for Proportion',
        'question_type': 'numerical_question',
        'question_text': '''In a survey of 500 voters, 300 indicated they support a new policy. Calculate the length of the 90% confidence interval for the true proportion of voters who support the policy. Provide your answer as a decimal rounded to four decimal places.''',
        'points_possible': 1,
        'answers': [
            {
                'numerical_answer_type': 'exact_answer',
                'answer_exact': '0.0720'  # Length = upper bound - lower bound = 0.636 - 0.564 = 0.0720
            }
        ]
    },
    {
        'question_name': 'Finding Critical Value for 99% Confidence Level',
        'question_type': 'numerical_question',
        'question_text': '''What is the critical value \( z^* \) for a 99% confidence interval? Provide your answer rounded to three decimal places.''',
        'points_possible': 1,
        'answers': [
            {
                'numerical_answer_type': 'exact_answer',
                'answer_exact': '2.576'  # Correct answer
            }
        ]
    },
    {
        'question_name': 'Calculating Margin of Error for Proportion',
        'question_type': 'numerical_question',
        'question_text': '''A sample proportion \( \hat{p} \) is 0.45 with a sample size of 200. The confidence level is 95%. Calculate the margin of error (ME) for the confidence interval. Provide your answer rounded to four decimal places.''',
        'points_possible': 1,
        'answers': [
            {
                'numerical_answer_type': 'exact_answer',
                'answer_exact': '0.0690'  # ME = z* × SE = 1.96 × sqrt(0.45*0.55/200) ≈1.96 × 0.0352 ≈0.0690
            }
        ]
    },
    {
        'question_name': 'Calculating Length of Confidence Interval for Proportion (Word Problem)',
        'question_type': 'numerical_question',
        'question_text': '''A marketing firm wants to estimate the proportion of customers who prefer their new product. They surveyed 300 customers, finding that 180 prefer the new product. Additionally, they recorded the customers' ages and purchase history. Calculate the length of the 95% confidence interval for the true proportion of customers who prefer the new product. Provide your answer as a decimal rounded to four decimal places.''',
        'points_possible': 1,
        'answers': [
            {
                'numerical_answer_type': 'exact_answer',
                'answer_exact': '0.1110'  # Length = 2 * ME = 2 * (1.96 * sqrt(0.6*0.4/300)) ≈2 * 0.0555 =0.1110
            }
        ]
    }
]

In [8]:
# Combine all questions, including the essay question
questions = multiple_choice_questions + numerical_questions + essay_questions

# Add questions to the quiz
for question in questions:
    create_question_url = f'{API_URL}/api/v1/courses/{COURSE_ID}/quizzes/{quiz_id}/questions'

    # Base question data
    question_data = {
        'question': {
            'question_name': question['question_name'],
            'question_type': question['question_type'],
            'question_text': question['question_text'],
            'points_possible': question.get('points_possible', 1)
            # Do not include 'answers' here
        }
    }

    # Include 'answers' only if they exist and the question is not an essay question
    if 'answers' in question and question['question_type'] != 'essay_question':
        question_data['question']['answers'] = question['answers']

    # Include 'general_feedback' if it exists (for essay questions)
    if 'general_feedback' in question:
        question_data['question']['general_feedback'] = question['general_feedback']

    # Send the request to create the question
    response = requests.post(create_question_url, headers=headers, json=question_data)

    if response.status_code == 201 or response.status_code == 200:
        print(f"Added question '{question['question_name']}' successfully.")
    else:
        print(f"Failed to add question '{question['question_name']}'")
        print(f"Status Code: {response.status_code}")
        print(response.text)

Added question 'Critical Value for 95% Confidence Level' successfully.
Added question 'Precondition for Z-Interval for Proportion' successfully.
Added question 'Standard Error for Population Proportion' successfully.
Added question 'Margin of Error Calculation' successfully.
Added question 'Point Estimate of Confidence Interval' successfully.
Added question 'Identifying Appropriate Confidence Interval Type' successfully.
Added question 'Identifying Appropriate Confidence Interval Type (Word Problem)' successfully.
Added question 'Calculating Standard Error for Mean' successfully.
Added question 'Calculating Length of Confidence Interval for Proportion' successfully.
Added question 'Finding Critical Value for 99% Confidence Level' successfully.
Added question 'Calculating Margin of Error for Proportion' successfully.
Added question 'Calculating Length of Confidence Interval for Proportion (Word Problem)' successfully.
